In [ ]:
# Import required packages
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from shutil import copy,move
from distutils.dir_util import copy_tree
import re

In [ ]:
for i in glob('16qubit/*/*_results.json'):
    df = pd.concat([pd.read_csv(j)[['Iteration','Cost']].dropna() for j in sorted(glob(os.path.join(os.path.dirname(i),'*log.csv')))]).sort_values(by='Iteration')
    plt.plot(df['Iteration'],df['Cost'])
    plt.title(os.path.basename(os.path.dirname(i)))
    plt.show()    

In [ ]:
for i in glob('5qubit/*/model_log.csv'):
    name = os.path.basename(os.path.dirname(i))
    df = pd.read_csv(i)[['Iteration','Cost']].dropna()
    plt.plot(df['Iteration'],df['Cost'])
    plt.title(os.path.basename(os.path.dirname(i)))
    plt.show()

In [ ]:
# Use a regex pattern to split the string at each date
pattern = r'\b[A-Z][a-z]{2} [A-Z][a-z]{2} \d{2} \d{2}:\d{2}:\d{2} \d{4}\b'


In [ ]:
stackeddf=[]
for i in glob("./5qubit/*/*results.json"):
    name = os.path.basename(os.path.dirname(i))
    with open(i) as f:
        data = json.load(f)
    data['encoder']=name.split('_')[0]
    data['ansatz']=name.split('_')[1]
    stackeddf.append(pd.DataFrame.from_dict(data).rename(index={0:name}))
print(len(stackeddf))
stackeddf = pd.concat(stackeddf).rename(columns={'MAE_train':'Train MAE','MAE_test':'Test MAE'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train MAE','Test MAE']).sort_values(by=['encoder','ansatz'])

g = sns.catplot(
    data=stackeddf, y="ansatz", x="value", col="encoder",hue="variable",
    kind="bar", height=5, aspect=.9,palette=sns.color_palette('Paired',2),col_wrap=3,legend_out=True)

g.set_titles("{col_name}")
# g.set_axis_labels("", "R$^{2}$")
g.set_axis_labels("", "MAE (kcal/mol)")
# g.set(ylim=(-1.1, 1.1))
# g.fig.suptitle(f"16 Qubit BSE Re-upload Depth (RUD) and Ansatz Layer (AL)")
g.tight_layout()
plt.show()

In [ ]:
stackeddf=[]
for i in glob("./16qubit/*/*results.json"):
    name = os.path.basename(os.path.dirname(i))
    with open(i) as f:
        data = json.load(f)
    data['encoder']=name.split('_')[0]
    data['ansatz']=name.split('_')[1]
    stackeddf.append(pd.DataFrame.from_dict(data).rename(index={0:name}))
print(len(stackeddf))
df = pd.concat(stackeddf)
stackeddf = df.rename(columns={'MAE_train':'Train MAE','MAE_test':'Test MAE'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train MAE','Test MAE']).sort_values(by=['encoder','ansatz'])

g = sns.catplot(
    data=stackeddf, y="ansatz", x="value", col="encoder",hue="variable",
    kind="bar", height=5, aspect=.9,palette=sns.color_palette('Paired',2),col_wrap=3,legend_out=True)

g.set_titles("{col_name}")
# g.set_axis_labels("", "R$^{2}$")
g.set_axis_labels("", "MAE (kcal/mol)")
idx=0
# Iterate over each Axes in the FacetGrid to add labels
for axidx,ax in enumerate(g.axes.flat):
    # Loop over each bar within the axis
    for pidx,p in enumerate(ax.patches):
        
        # Get the center of the bar's x position and the height for the label
        x = p.get_y()
        y = p.get_y()
        print(x,y)
        # Set label with the value
        ax.text(x, y, f"{float(stackeddf['value'].iloc[idx]):.2f}", ha='center', va='center', fontsize=8, color='black', rotation=0)
        idx+=1
# # extract the matplotlib axes_subplot objects from the FacetGrid
# ax = g.facet_axis(0, 0)  # or ax = g.axes.flat[0]

# # iterate through the axes containers
# for c in ax.containers:
#     labels = [f'{(v.get_height()):.1e}' for v in c]
#     ax.bar_label(c, labels=labels, label_type='edge')
# g.set(ylim=(-1.1, 1.1))
# g.fig.suptitle(f"16 Qubit BSE Re-upload Depth (RUD) and Ansatz Layer (AL)")
g.tight_layout()
plt.show()

In [ ]:
df.sort_values(by=['MAE_train','MAE_test'])